In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
df = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
df.shape

(12120, 6)

In [3]:
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [5]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size=0.20, random_state=42, stratify=df['language'])

In [6]:
print(train.shape)
train.head()

(9696, 6)


,id,premise,hypothesis,lang_abv,language,label
661,2da1be6f81,Hazmi的室友记得他在这段时间不知道为什么去了圣地亚哥机场。,哈兹米的室友从未听说过圣地亚哥机场。,zh,Chinese,2
1166,237606efd9,Skeat จะไม่ในกรณีนี้ ปฏิเสธบันทึกนี้ และ ซ้ำกั...,สเกทจะไม่ให้ความสนใจใดๆต่อข้อความ,th,Thai,2
11812,5777575ffc,"Το εξαιρετικό δοκίμιο του Jacob Weisberg, Car ...",Ο Weisberg έγραψε για τα σκυλιά.,el,Greek,2
2679,1c99e3d69d,uh-huh you can't do that in a skirt poor thing,You cannot do that wearing a skirt.,en,English,0
8814,5ddbe24caf,But the most sustained assault on Orientalism ...,They were attack from the right.,en,English,2


In [7]:
print(val.shape)
val.head()

(2424, 6)


,id,premise,hypothesis,lang_abv,language,label
10810,1c7273a301,"There's one thing, he thought to himself, ""the...",He thought to himself that they can continue s...,en,English,2
8421,c6e980fab1,"Това означава, че всички молекулярни елементи ...",Не можете да добавяте неща с постоянно темпо.,bg,Bulgarian,2
7844,379553618d,The track continues past the necropolis to an ...,The path leads to an amphitheater that was inf...,en,English,0
369,c9564000f9,"um, nina mtoto mmoja msichana mdogo ambaye ana...","nina mtoto mmoja, binti wa miezi kumi na nane",sw,Swahili,0
3401,ca2e383db3,"In the summer, the Sultan's Pool, a vast outdo...",Most rock concerts take place in the Sultan's ...,en,English,1


In [12]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train, split="train")
val_ds = Dataset.from_pandas(val, split="test")

train_ds, val_ds

(Dataset({
     features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label', '__index_level_0__'],
     num_rows: 9696
 }),
 Dataset({
     features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label', '__index_level_0__'],
     num_rows: 2424
 }))

In [13]:
from transformers import AutoTokenizer

checkpoint = "google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(row):
    return tokenizer(row["language"], row["premise"], row["hypothesis"], truncation=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
train_tok = train_ds.map(tokenize_function, batched=True)
val_tok = val_ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/9696 [00:00<?, ? examples/s]

Map:   0%|          | 0/2424 [00:00<?, ? examples/s]

In [51]:
train_tok

Dataset({
    features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 9696
})

In [57]:
train_tok[3]

{'id': '1c99e3d69d',
 'premise': "uh-huh you can't do that in a skirt  poor thing",
 'hypothesis': 'You cannot do that wearing a skirt.',
 'lang_abv': 'en',
 'language': 'English',
 'label': 0,
 '__index_level_0__': 2679,
 'input_ids': [101,
  11616,
  102,
  189,
  10237,
  118,
  26506,
  10237,
  13028,
  10944,
  112,
  188,
  10149,
  10189,
  10106,
  169,
  40122,
  10976,
  23247,
  40414,
  102],
 'token_type_ids': [0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [101,
  11065,
  25259,
  10149,
  10189,
  56157,
  169,
  40122,
  10976,
  119,
  102]}

In [17]:
from transformers import AutoTokenizer, DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="/kaggle/working/", 
                                  report_to="none",
                                  num_train_epochs=1
                                 )

columns_to_select = ['input_ids', 'token_type_ids', 'attention_mask', 'label']

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_tok.select_columns(columns_to_select),
    eval_dataset=val_tok.select_columns(columns_to_select),
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,1.064800


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=606, training_loss=1.0614978425180164, metrics={'train_runtime': 192.4624, 'train_samples_per_second': 50.379, 'train_steps_per_second': 3.149, 'total_flos': 386554722113952.0, 'train_loss': 1.0614978425180164, 'epoch': 1.0})

In [53]:
predictions = trainer.predict(val_tok.select_columns(columns_to_select))
print(predictions.predictions.shape, predictions.label_ids.shape)
preds = np.argmax(predictions.predictions, axis=-1)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


(2424, 3) (2424,)


In [47]:
!pip install evaluate --quiet

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
import evaluate

metric = evaluate.load("accuracy")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.27433993399339934}